# IMDB Rating Prediction - Random Forest

Bu notebook'ta IMDB film verisini kullanarak **rating** değerini
Random Forest ile tahmin ediyoruz. Özellikler:

- Sayısal: `votes`, `runtime`
- Kategorik: `genre` (multi-hot encoding), `director` (target encoding),`stars` (multi-value target encoding)
- Metin: `description` (TF-IDF)

Model:
- `RandomForestRegressor`

Değerlendirme:
- Primary metric: **RMSE**
- Secondary metrics: **MSE**, **R²**
- Değerlendirme yöntemi:
  - Train set üzerinde **5-fold cross validation** (GridSearchCV ile)
  - Ayrı tutulan **test seti** üzerinde final performans (RMSE, MSE, R²)


## 1. Data Loading, Cleaning & Data Info

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
MIN_VOTES = 100
MIN_RUNTIME = 60
SMOOTH_FACTOR = 5

data_path = "/content/imbd_data/IMDB_cleaned.csv"   # data_clean.py çıktısı
df = pd.read_csv(data_path)


df = df[df['votes'] > MIN_VOTES]
df = df[df['runtime'] >= MIN_RUNTIME]

print("[INFO] Data size:", df.shape)
df.head()


[INFO] Data size: (39238, 8)


,movie,genre,runtime,rating,stars,description,votes,director
0,Mission: Impossible - Dead Reckoning Part One,"Action, Adventure, Thriller",163,8.0,"Tom Cruise, Hayley Atwell, Ving Rhames, Simon ...",Ethan Hunt and his IMF team must track down a ...,106759,Christopher McQuarrie
1,Sound of Freedom,"Action, Biography, Drama",131,7.9,"Jim Caviezel, Mira Sorvino, Bill Camp, Cristal...",The incredible true story of a former governme...,41808,Alejandro Monteverde
2,They Cloned Tyrone,"Action, Comedy, Mystery",122,6.7,"John Boyega, Jamie Foxx, Teyonah Parris, Kiefe...",A series of eerie events thrusts an unlikely t...,14271,Juel Taylor
3,The Flash,"Action, Adventure, Fantasy",144,6.9,"Ezra Miller, Michael Keaton, Sasha Calle, Mich...",Barry Allen uses his super speed to change the...,126445,Andy Muschietti
4,Transformers: Rise of the Beasts,"Action, Adventure, Sci-Fi",127,6.1,"Anthony Ramos, Dominique Fishback, Luna Lauren...","During the 90s, a new faction of Transformers ...",62180,Steven Caple Jr.


In [ ]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 39238 entries, 0 to 60914
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   movie        39238 non-null  object 
 1   genre        39238 non-null  object 
 2   runtime      39238 non-null  int64  
 3   rating       39238 non-null  float64
 4   stars        39238 non-null  object 
 5   description  39238 non-null  object 
 6   votes        39238 non-null  int64  
 7   director     39238 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 2.7+ MB


## 2. Genre Encoding (Multi-hot)

Bu adımda:

- `genre` sütunundaki stringleri sadeleştiriyoruz (boşlukları kaldırıyoruz).
- `str.get_dummies(sep=",")` kullanarak her tür için ayrı bir dummy sütunu (multi-hot encoding) oluşturuyoruz.
- Orijinal `genre` sütununu drop edip yerine bu dummy sütunlarını ekliyoruz.

Bu encoding, hedef değişkeni (**rating**) kullanmadığı için
**train/test split'inden önce** yapmak güvenli (data leakage oluşturmaz).


In [ ]:
df_model = df.copy()

# genre içindeki boşlukları kaldır
df_model["genre"] = df_model["genre"].astype(str).str.replace(" ", "", regex=False)

# multi-hot encoding
genres_encoded = df_model["genre"].str.get_dummies(sep=",")

print("[INFO] Tür sayısı:", genres_encoded.shape[1])

# genre sütununu kaldır, encoded sütunları ekle
df_model = pd.concat([df_model.drop(columns=["genre"]), genres_encoded], axis=1)

df_model.head()


[INFO] Tür sayısı: 26


,movie,runtime,rating,stars,description,votes,director,Action,Adventure,Animation,...,Mystery,News,Reality-TV,Romance,Sci-Fi,Sport,Talk-Show,Thriller,War,Western
0,Mission: Impossible - Dead Reckoning Part One,163,8.0,"Tom Cruise, Hayley Atwell, Ving Rhames, Simon ...",Ethan Hunt and his IMF team must track down a ...,106759,Christopher McQuarrie,1,1,0,...,0,0,0,0,0,0,0,1,0,0
1,Sound of Freedom,131,7.9,"Jim Caviezel, Mira Sorvino, Bill Camp, Cristal...",The incredible true story of a former governme...,41808,Alejandro Monteverde,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,They Cloned Tyrone,122,6.7,"John Boyega, Jamie Foxx, Teyonah Parris, Kiefe...",A series of eerie events thrusts an unlikely t...,14271,Juel Taylor,1,0,0,...,1,0,0,0,0,0,0,0,0,0
3,The Flash,144,6.9,"Ezra Miller, Michael Keaton, Sasha Calle, Mich...",Barry Allen uses his super speed to change the...,126445,Andy Muschietti,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,Transformers: Rise of the Beasts,127,6.1,"Anthony Ramos, Dominique Fishback, Luna Lauren...","During the 90s, a new faction of Transformers ...",62180,Steven Caple Jr.,1,1,0,...,0,0,0,0,1,0,0,0,0,0


## 3. X ve y Oluşturma

- Hedef değişken (y) = `rating`
- Özellikler (X):
  - Sayısal + genre sütunları: `votes`, `runtime` + tüm genre dummy'leri
  - Kategorik sütunlar: `director`, `stars` (bunları ileride target-encode edeceğiz)
  - Metin sütunu: `description`


In [ ]:
target_col = "rating"
y = df_model[target_col]

# genre dummy sütunlarının isimleri
genre_cols = list(genres_encoded.columns)

# temel sayısal sütunlar
numeric_base = ["votes", "runtime"]

# sayısal + genre sütunlarının tamamı
numeric_features_full = numeric_base + genre_cols

# target encoding yapılacak kategorik sütunlar
director_col = "director"
stars_col = "stars"

# metin sütunu
text_feature = "description"

# X: numeric + categorical + text sütunları
X = df_model[numeric_features_full + [director_col, stars_col, text_feature]]

print("[INFO] X şekli:", X.shape)
print("[INFO] y şekli:", y.shape)

X.head()


[INFO] X şekli: (39238, 31)
[INFO] y şekli: (39238,)


,votes,runtime,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,...,Romance,Sci-Fi,Sport,Talk-Show,Thriller,War,Western,director,stars,description
0,106759,163,1,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,Christopher McQuarrie,"Tom Cruise, Hayley Atwell, Ving Rhames, Simon ...",Ethan Hunt and his IMF team must track down a ...
1,41808,131,1,0,0,1,0,0,0,1,...,0,0,0,0,0,0,0,Alejandro Monteverde,"Jim Caviezel, Mira Sorvino, Bill Camp, Cristal...",The incredible true story of a former governme...
2,14271,122,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,Juel Taylor,"John Boyega, Jamie Foxx, Teyonah Parris, Kiefe...",A series of eerie events thrusts an unlikely t...
3,126445,144,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,Andy Muschietti,"Ezra Miller, Michael Keaton, Sasha Calle, Mich...",Barry Allen uses his super speed to change the...
4,62180,127,1,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,Steven Caple Jr.,"Anthony Ramos, Dominique Fishback, Luna Lauren...","During the 90s, a new faction of Transformers ..."


## 4. Train / Test Split

Veriyi:

- %80 **train**
- %20 **test**

şeklinde ayıracağız.

Önemli Not:  
`director` ve `stars` için target encoding işlemini **sadece train set** üzerinden
öğreneceğiz; böylece test setin rating bilgisi encoding aşamasına sızmayacak (Target encoding -> Bir kategori, o kategoriye ait örneklerdeki hedef değişkenin ortalama değeri ile kodlanır.)
→ **data leakage** engellenmiş olacak.


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)

print("[INFO] Train:", X_train.shape)
print("[INFO] Test :", X_test.shape)


[INFO] Train: (31390, 31)
[INFO] Test : (7848, 31)


## 5. Director & Stars için Target Encoding (Sadece Train Üzerinden)

Bu bölümde iki farklı encoding yapacağız:

1. **Director (tek değer) için:**
   - Train set üzerinde `director` → ortalama `rating`
   - Görülmeyen director'ler için global ortalama

2. **Stars (çoklu değer) için:**
   - Her filmde `stars` şu formda: `"Oyuncu1, Oyuncu2, Oyuncu3"`
   - Önce bu string'i listeye çeviriyoruz: `["Oyuncu1", "Oyuncu2", "Oyuncu3"]`
   - Train set üzerinde her oyuncu için ortalama rating hesaplıyoruz.
   - Bir filmin star değeri = o filmdeki tüm oyuncuların ortalama ratinglerinin ortalaması
   - Test set'te görülmeyen oyuncular için global ortalama kullanıyoruz.

Bu sayede:

- `director` ve `stars` sütunları sayısallaşmış oluyor.
- Encoding işlemi sadece train verisini "gördüğü" için test bilgisi sızmıyor (leakage yok).


In [ ]:
def target_encode_director(X_train, X_test, y_train, director_col):
    """
    director için target encoding:
    - Train üzerinde director -> ortalama rating hesaplar
    - Train ve test'te bu mapping ile encode eder
    - Görülmeyen director'ler için global ortalama kullanır
    """
   # 1) Train kopyası + hedefi ekle
temp_train_dir = X_train.copy()
temp_train_dir["rating"] = y_train

# 2) Global mean
global_mean = temp_train_dir["rating"].mean()
print(f"[INFO] Global mean rating (train): {global_mean:.4f}")

# 3) Director bazında count ve mean
agg_dir = temp_train_dir.groupby("director")["rating"].agg(["count", "mean"])
counts = agg_dir["count"]
means = agg_dir["mean"]

# 4) Smoothed mean
dir_smooth = (counts * means + SMOOTH_FACTOR * global_mean) / (counts + SMOOTH_FACTOR)

# 5) Mapping'i uygula
X_train_enc = X_train.copy()
X_test_enc  = X_test.copy()

X_train_enc["director_te"] = X_train_enc["director"].map(dir_smooth).fillna(global_mean)
X_test_enc["director_te"]  = X_test_enc["director"].map(dir_smooth).fillna(global_mean)

# 6) Artık orijinal director sütununa ihtiyacımız yok
X_train_enc = X_train_enc.drop(columns=["director"])
X_test_enc  = X_test_enc.drop(columns=["director"])

print("[INFO] Smoothed target encoding applied for director → director_te")
X_train_enc.head()



[INFO] Global mean rating (train): 5.9229
[INFO] Smoothed target encoding applied for director → director_te


,votes,runtime,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,...,Romance,Sci-Fi,Sport,Talk-Show,Thriller,War,Western,stars,description,director_te
25825,805,75,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,"Rhonda Griffin, Justin Lauer, Bill Moynihan, K...","Mad scientist brings Dracula, the Wolfman, the...",4.621191
56661,1788,101,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,"Dean Martin, Alain Delon, Rosemary Forsyth, Jo...","In 1845 Texas, a Louisiana belle is courted by...",6.369658
27297,934,118,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,"Michal Zebrowski, Robert Zoledziewski, Jan Eng...","September 1st, 1939. German battleship Schlesw...",5.769088
41797,135,91,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,"Roberto Camardiel, Pedro Porcel, George Rigaud...",Life events around a city street and its resid...,6.102075
24761,1557,105,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,"Allan Katz, Corey Parker, Cindy Williams, Melo...",In this version of The Hunchback of Notre Dame...,6.201321


In [ ]:
def smoothed_target_encoding_stars_multi(X_train, X_test, y_train, stars_col, m):
    """
    Multi-value stars için smoothed target encoding:
    - "Oyuncu1, Oyuncu2" gibi stringleri listeye çevirir
    - Train set üzerinde her star için (count, mean) hesaplar
    - Smoothed mean: (count*mean + m*global_mean) / (count + m)
    - Bir filmin stars değeri = o filmdeki tüm star'ların smoothed mean'lerinin ortalaması
    - Test set'te görülmeyen star'lar için global mean kullanır
    """
    X_train = X_train.copy()
    X_test = X_test.copy()
    y_train = pd.Series(y_train)

    # 1) Global mean
    global_mean = y_train.mean()

    # 2) star string → liste
    X_train["stars_list"] = X_train[stars_col].apply(
        lambda x: [s.strip() for s in str(x).split(",")]
    )
    X_test["stars_list"] = X_test[stars_col].apply(
        lambda x: [s.strip() for s in str(x).split(",")]
    )

    # 3) Train'de her star için tüm rating'leri topla (explode ile)
    df_exploded = pd.DataFrame({
        "stars": X_train["stars_list"],
        "rating": y_train
    }).explode("stars")

    agg_stars = df_exploded.groupby("stars")["rating"].agg(["count", "mean"])
    counts = agg_stars["count"]
    means = agg_stars["mean"]

    # 4) Smoothed mean
    star_smooth = (counts * means + m * global_mean) / (counts + m)

    # 5) Bir film için stars_list'ten tek bir smoothed değer üret
    def encode_star_list(stars):
        vals = [star_smooth.get(s, global_mean) for s in stars]
        return np.mean(vals) if len(vals) > 0 else global_mean

    X_train["stars_te"] = X_train["stars_list"].apply(encode_star_list)
    X_test["stars_te"]  = X_test["stars_list"].apply(encode_star_list)

    # 6) Geçici sütunları drop et
    X_train = X_train.drop(columns=[stars_col, "stars_list"])
    X_test  = X_test.drop(columns=[stars_col, "stars_list"])

    print("[INFO] Smoothed target encoding applied for stars → stars_te")

    return X_train, X_test


# Fonksiyonu bizim mevcut X_train_enc / X_test_enc üstüne uyguluyoruz
X_train_enc, X_test_enc = smoothed_target_encoding_stars_multi(
    X_train_enc, X_test_enc, y_train, stars_col="stars", m=SMOOTH_FACTOR
)

print("[INFO] Encoded train shape:", X_train_enc.shape)
print("[INFO] Encoded test  shape:", X_test_enc.shape)
X_train_enc.head()

[INFO] Smoothed target encoding applied for stars → stars_te
[INFO] Encoded train shape: (31390, 31)
[INFO] Encoded test  shape: (7848, 31)


,votes,runtime,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,...,Romance,Sci-Fi,Sport,Talk-Show,Thriller,War,Western,description,director_te,stars_te
25825,805,75,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,"Mad scientist brings Dracula, the Wolfman, the...",4.621191,5.669306
56661,1788,101,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,"In 1845 Texas, a Louisiana belle is courted by...",6.369658,6.149206
27297,934,118,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,"September 1st, 1939. German battleship Schlesw...",5.769088,5.857162
41797,135,91,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,Life events around a city street and its resid...,6.102075,5.888994
24761,1557,105,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,In this version of The Hunchback of Notre Dame...,6.201321,5.742764


## 6. Numeric ve Text Özelliklerini Ayırma

Artık:

- `description` dışındaki tüm özellikler sayısal (votes, runtime, genre dummies, director_te, stars_te).
- `description` ise metin sütunu.

TF-IDF vektörleştirmesini sadece `description` üzerinde uygulayacağız.
Diğer tüm sayısal sütunlar olduğu gibi Random Forest'a gidecek.


In [ ]:
# text sütunu dışındaki tüm sütunlar numeric olarak kabul edilecek
numeric_features = [col for col in X_train_enc.columns if col != text_feature]

print("[INFO] Numeric feature sayısı:", len(numeric_features))
print("[INFO] Text feature:", text_feature)


[INFO] Numeric feature sayısı: 30
[INFO] Text feature: description


## 7. TF-IDF + Random Forest Pipeline ve 5-Fold Cross Validation

Bu bölümde:

- `description` → **TF-IDF** (`TfidfVectorizer`) ile vektörleştirilecek.
- Numeric özellikler (votes, runtime, genre dummies, director_te, stars_te) → doğrudan geçecek.
- `ColumnTransformer` ile numeric + text birleşik feature space oluşturacağız.
- Sparse TF-IDF çıktısını `FunctionTransformer` ile dense formata çevireceğiz (Random Forest dense bekliyor).
- Model: `RandomForestRegressor`
- `GridSearchCV` ile:
  - 5-fold CV
  - Primary metric: **RMSE** (`neg_root_mean_squared_error`)
  - Hiperparametre taraması (n_estimators, max_depth, min_samples_leaf)


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

# 1) Text -> TF-IDF
text_transformer = TfidfVectorizer(
    max_features=1000,
    stop_words="english"
)

# 2) ColumnTransformer: numeric + text
preprocessor = ColumnTransformer(
    transformers=[
        ("num", "passthrough", numeric_features),
        ("text", text_transformer, text_feature)
    ],
    remainder="drop"
)

# 3) Sparse -> dense (RF için)
to_dense = FunctionTransformer(
    lambda x: x.toarray() if hasattr(x, "toarray") else np.asarray(x),
    accept_sparse=True
)

# 4) Random Forest modeli
rf = RandomForestRegressor(
    random_state=42,
    n_jobs=-1
)

# 5) Tüm pipeline
rf_pipeline = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("to_dense", to_dense),
    ("model", rf)
])

# 6) Parametre grid'i
param_grid = {
    "model__n_estimators": [200],        # sabit
    "model__max_depth": [None, 20],      # 2 değer
    "model__min_samples_leaf": [1, 2, 4],   # 2 değer
}
# 1 * 2 * 3 = 6 kombinasyon

# 7) Grid search
grid_search = GridSearchCV(
    estimator=rf_pipeline,
    param_grid=param_grid,
    cv=5,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1,
    verbose=2
)

grid_search.fit(X_train_enc, y_train)


Fitting 5 folds for each of 6 candidates, totalling 30 fits


KeyboardInterrupt: 

## 8. Cross Validation Sonuçları

Burada:

- GridSearchCV'nin bulduğu en iyi hiperparametreleri,
- 5-fold CV sonucunda elde edilen **en iyi ortalama RMSE** değerini göreceğiz.


In [ ]:
print("========== GRID SEARCH SONUÇLARI ==========")
print("En iyi hiperparametreler:", grid_search.best_params_)

best_cv_rmse = -grid_search.best_score_
print(f"5-Fold CV En İyi Ortalama RMSE: {best_cv_rmse:.4f}")


========== GRID SEARCH SONUÇLARI ==========
En iyi hiperparametreler: {'model__max_depth': None, 'model__max_features': 'sqrt', 'model__min_samples_leaf': 1, 'model__n_estimators': 200}
5-Fold CV En İyi Ortalama RMSE: 0.4788


## 9. Değişkenlerin Önemini Bulma
Şimdi RF modelimiz için değişkenlerin önem sırasını görelim:

In [ ]:
best_model = grid_search.best_estimator_

# 1) RF modelini pipeline içinden çek
rf_model = best_model.named_steps["model"]

# 2) TF-IDF transformerı çek
tfidf = best_model.named_steps["preprocess"].named_transformers_["text"]

# 3) TF-IDF feature isimlerini al
tfidf_features = tfidf.get_feature_names_out()

# 4) Numeric feature listesi + TF-IDF
all_features = numeric_features + list(tfidf_features)

print("[INFO] Toplam feature sayısı:", len(all_features))
print("[INFO] RF importance sayısı:", len(rf_model.feature_importances_))

# 5) Feature importance dataframe
fi = pd.DataFrame({
    "feature": all_features,
    "importance": rf_model.feature_importances_
}).sort_values(by="importance", ascending=False)

fi.head(10)   # ilk 10 özelliği göster


[INFO] Toplam feature sayısı: 1032
[INFO] RF importance sayısı: 1032


,feature,importance
31,stars_te,0.308291
30,director_te,0.277884
1,runtime,0.032004
0,votes,0.026907
16,Horror,0.023926
9,Documentary,0.018708
24,Short,0.009512
23,Sci-Fi,0.007592
2,Action,0.006445
10,Drama,0.006182


## 10. Test Set Üzerinde Final Değerlendirme

Artık:

- `grid_search.best_estimator_` bize en iyi hiperparametrelere sahip Random Forest pipeline'ını veriyor.
- Bu modeli, en başta ayırdığımız **test seti** üzerinde değerlendiriyoruz.

Hesaplanan metrikler:

- **RMSE (primary metric)**
- **MSE**
- **R²**


In [ ]:
best_model = grid_search.best_estimator_

# Test set tahminleri
y_pred = best_model.predict(X_test_enc)

# Secondary: MSE
mse = mean_squared_error(y_test, y_pred)
# Primary: RMSE
rmse = np.sqrt(mse)
# Secondary: R²
r2 = r2_score(y_test, y_pred)

print("========== TEST SET PERFORMANSI ==========")
print(f"RMSE (Primary):   {rmse:.4f}")
print(f"MSE  (Secondary): {mse:.4f}")
print(f"R²   (Secondary): {r2:.4f}")

========== TEST SET PERFORMANSI ==========
RMSE (Primary):   1.0785
MSE  (Secondary): 1.1631
R²   (Secondary): 0.3728
